## ДЗ2 часть 1
### Такташева Катя, БКЛ182

По какой-то причине после запуска проверки Jupyter в один момент просто перестал выводить хоть что-либо в этой тетрадке... При копии тетрадки и запуске без проверки на пеп все нормально работает, поэтому закомментила, может это у меня что на ноуте. До этого все нормально прошло проверку и работало отлично...

In [26]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

Импортируем нужные модули:

In [27]:
import json
from pymystem3 import Mystem
m = Mystem()

### Задание 1: Текст книги

Запишем текст книги в переменную text:

In [28]:
with open('book.txt', 'r', encoding='utf-8') as fid:
    text = fid.read()

### Задание 2: MyStem

In [29]:
%%time
lemmas_ms = m.analyze(text)

CPU times: user 1.07 s, sys: 537 ms, total: 1.6 s
Wall time: 11.5 s


Преобразуем полученные данные в формат .json и запишем в файл 'lemmas_mystem.json':

In [56]:
with open('lemmas_mystem.json', 'w', encoding='utf-8') as fid:
    json.dump(lemmas_ms, fid, ensure_ascii = False)

### Задание 3: PyMorphy
Импортируем нужные модули:

In [31]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Преобразуем текст в токены для анализа:

In [32]:
tokens = word_tokenize(text)
lemmas = []

In [33]:
%%time
for token in tokens:
    lemma = morph.parse(token)
    lemmas.append(lemma)

CPU times: user 4.89 s, sys: 75.8 ms, total: 4.96 s
Wall time: 5.11 s


"Вытащим" из полученных данных информацию о словоформе, а также всех вариантах разбора этой словоформы, и добавим в словарь лексему, часть речи и число каждого из них.

**P.S.:** Далее при создании словаря наткнулась на проблему, что некоторые варианты парсинга одного слова могут различаться буквой е/ё (например, один из вариантов разбора слова 'весны' - 'вёсны'), поэтому в ключе заменим все 'ё' на 'е'.

In [34]:
dic = dict()
lemmas_pm = []
for i in range(len(lemmas)):
    dic = dict()
    dic[lemmas[i][0].word.replace('ё', 'е')] = []
    for wordform in lemmas[i]:
        dic[wordform.word.replace('ё', 'е')].append({
            'normal_form': wordform.normal_form,
            'pos': wordform.tag.POS,
            'number': wordform.tag.number
        })
    lemmas_pm.append(dic)

Преобразуем полученные данные в формат .json и запишем в файл 'lemmas_pymorphy.json':

In [55]:
with open('lemmas_pymorphy.json', 'w', encoding='utf-8') as fid:
    json.dump(lemmas_pm, fid, ensure_ascii = False)

### Задание 4.1: Части речи
**NB:** Для анализа будут использованы данные, распарсенные Mystem, потому что они более точные. Стоп-слова не убираем.

Импортируем Counter:

In [36]:
from collections import Counter

Преобразуем данные из формата json в объект питона:

In [37]:
parced_data = []
for line in open('lemmas_mystem.json', encoding='utf-8'):
    parced_data.extend(json.loads(line))

Посчитаем количество каждой из частей речи в тексте:

In [38]:
pos_list = []
for line in parced_data:
    if 'analysis' in line.keys():
        if line['analysis'] != []:
            pos = line['analysis'][0]['gr']
            pos = pos.replace('=', ',').split(',')[0]
            pos_list.append(pos)
counter_pos = Counter(pos_list)

Создадим строку frequency, в которую запишем все нужные для вывода даные: 

In [39]:
frequency = 'POS:\tДоля слов:\n'
for pos, count in counter_pos.items():
    percent = count / len(pos_list)
    frequency += pos + '\t' + str(percent) + '\n'

#### Ответ:

In [40]:
print('Частотность слов по частям речи:', frequency, sep='\n\n')

Частотность слов по частям речи:

POS:	Доля слов:
S	0.2899892809388572
ANUM	0.0038013990032378193
PART	0.05455670604356138
CONJ	0.08207264650304444
SPRO	0.06148541876167218
V	0.17737283546793675
PR	0.10734531952747726
A	0.09134408186268551
ADVPRO	0.020145204601460888
APRO	0.03876542937022753
ADV	0.06354082636226006
NUM	0.007801708419435757
INTJ	0.0017570419811477132
COM	2.2101156995568717e-05



### Задание 4.2: Топ-20 глаголов и наречий
Пройдемся по словарю и добавим в списки глаголов и наречий лексемы. Для деления заменим знак =, используемый для отсечения части речи и ее неизменяемых признаков.

In [41]:
adv_list = []
verb_list = []
for line in parced_data:
    if 'analysis' in line.keys():
        if line['analysis'] != []:
            pos = line['analysis'][0]['gr'].replace('=', ',').split(',')[0]
            if pos == 'V':
                verb_list.append(line['analysis'][0]['lex'])
            elif pos == 'ADV':
                adv_list.append(line['analysis'][0]['lex'])

Создадим строку avd_top, в которую запишем топ-20 наречий, встречающихся в тексте: 

In [42]:
adv_top = '№:\tЛексема:\tКол-во употреблений:\n'
i = 1
for word, count in Counter(adv_list).most_common(20):
    adv_top += str(i) + '\t' + word + '\t\t' + str(count) + '\n'
    i += 1

Аналогично для глаголов:

In [43]:
verb_top = '№:\tЛексема:\tКол-во употреблений:\n'
i = 1
for word, count in Counter(verb_list).most_common(20):
    verb_top += str(i) + '\t' + word + '\t\t' + str(count) + '\n'
    i += 1

#### Ответ:

In [44]:
print('Топ-20 глаголов:', verb_top, sep='\n\n', end='\n')
print('Топ-20 наречий:', adv_top, sep='\n\n', end='\n')

Топ-20 глаголов:

№:	Лексема:	Кол-во употреблений:
1	быть		1173
2	становиться		224
3	знать		217
4	сказать		200
5	мочь		177
6	говорить		127
7	давать		127
8	понимать		120
9	хотеть		110
10	видеть		96
11	спрашивать		87
12	выходить		86
13	стоять		84
14	думать		80
15	оставаться		77
16	оказываться		70
17	идти		69
18	сидеть		69
19	отвечать		67
20	взять		66

Топ-20 наречий:

№:	Лексема:	Кол-во употреблений:
1	еще		333
2	уже		150
3	теперь		134
4	много		104
5	очень		100
6	можно		97
7	хорошо		96
8	сейчас		87
9	вдруг		74
10	надо		73
11	конечно		69
12	сразу		65
13	более		61
14	совсем		60
15	скоро		58
16	далеко		57
17	снова		54
18	нужно		47
19	назад		46
20	сначала		46



### Задание 5: Топ-20 биграмм и триграмм
Создадим список лемм из текста:

In [45]:
words_lemmas = []
for word in parced_data:
    if 'analysis' in word.keys():
        if word['analysis'] != []:
            lemma = word['analysis'][0]['lex']
            words_lemmas.append(lemma)

Найдем все 2- и 3-граммы, объединим их в одном списке и создадим частотный словарь:

In [46]:
bigr = list(nltk.bigrams(words_lemmas))
trigr = list(nltk.trigrams(words_lemmas))
bigrams_counter = Counter(bigr)

Запишем топ-25 2-грамм в строку bigrams_top. Аналогично с 3-граммами.

In [51]:
bigrams_top = '№:\tКол-во:\t2-грамма:\n'
i = 1
for gram, count in bigrams_counter.most_common(25):
    bigrams_top += str(i) + '\t' + str(count) + '\t' + ' '.join(gram) + '\n'
    i += 1

In [52]:
trigrams_top = '№:\tКол-во:\t3-грамма:\n'
i = 1
for gram, count in Counter(trigr).most_common(25):
    trigrams_top += str(i) + '\t' + str(count) + '\t' + ' '.join(gram) + '\n'
    i += 1

#### Ответ:

In [53]:
print('Самые частотные 2-граммы:', bigrams_top, sep='\n\n')

Самые частотные 2-граммы:

№:	Кол-во:	2-грамма:
1	244	фон дорн
2	116	и не
3	106	то что
4	84	и в
5	74	потому что
6	73	я не
7	70	не быть
8	61	не знать
9	60	да и
10	55	не становиться
11	55	в этот
12	54	знать что
13	54	быть не
14	53	у я
15	51	что это
16	50	все равный
17	47	не мочь
18	47	и быть
19	47	у он
20	46	что я
21	45	он не
22	45	мы с
23	44	вы не
24	43	в самый
25	43	иосиф гурамович



При анализе 2-грамм можно заметить, что чамыми частотными являются имена героев произведения, а также сочетания в принципе частотные для русского языка - "не знать", "не быть", "не мочь" - и связки типа "потому что", "то что", "и не" и т.д.

In [54]:
print('Самые частотные 3-граммы:', trigrams_top, sep='\n\n')

Самые частотные 3-граммы:

№:	Кол-во:	3-грамма:
1	28	в самый дело
2	26	мы с вы
3	23	капитан фон дорн
4	21	и в самый
5	19	корнелиус фон дорн
6	16	о то что
7	16	к то же
8	13	в то что
9	12	не знать что
10	12	перед то как
11	12	в тот же
12	11	это и быть
13	11	так и не
14	11	мы с ты
15	11	библиотека иван грозный
16	10	в тот пора
17	10	знать что я
18	10	и без то
19	10	на самый дело
20	10	а то и
21	9	фон дорн и
22	9	в конец конец
23	9	ни за что
24	9	у он быть
25	9	у я быть



Получим схожие результаты: в топе находятся имя главного героя, а также частые русские триграммы: "в самом деле", "о том, что", "к тому же", "перед тем как" и т.д,  